In [1]:
import numpy as np
import pandas as pd
import glob
from scipy import stats
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import cv2

In [4]:
brodmann = pd.read_csv('../../Dataset/rna/brodmann_schaefer_0.5.csv')

In [6]:
SAaxis = pd.read_csv('../../Modules/human/Sensorimotor_Association_Axis_AverageRanks.csv')

In [ ]:
allf = glob.glob('../../Functional/derivative/xcp_d_bpd_allf_2/xcp_d/sub-BPD*/ses-01/func/sub-BPD*_ses-01_task-rest_run-*_space-MNI152NLin6Asym_atlas-Schaefer417_alff.tsv')
df_allf = pd.DataFrame()
sub = []
run = []
for x in allf:
    df = pd.read_csv(x, sep='\t')
    df_allf = pd.concat([df_allf, df])
    sub.append(os.path.basename(x).split('_')[0].replace('sub-', ''))
    run.append(int(os.path.basename(x).split('_')[3].replace('run-', '')))
df_allf['fsid'] = sub
df_allf['run'] = run
df_allf.to_csv('../derivatives/adult/allf.csv', index=False)

In [ ]:
allf = glob.glob('../../Functional/derivative/xcp_d_t1_allf_2/xcp_d/sub-CBPD*/ses-01/func/sub-CBPD*_ses-01_task-rest_run-*_space-MNI152NLin6Asym_atlas-Schaefer417_alff.tsv')
qc = pd.read_csv('../../Functional/derivative/xcp_d_t1_original/QA_revision.csv')
df_allf = pd.DataFrame()
sub = []
runs = []
for x in allf:
    sbj = os.path.basename(x).split('_')[0].replace('sub-', '')
    run = int(os.path.basename(x).split('_')[3].replace('run-', ''))
    if qc[qc['sub']==sbj][qc['run']==run].shape[0]>0:
        df = pd.read_csv(x, sep='\t')
        df_allf = pd.concat([df_allf, df])
        sub.append(sbj)
        runs.append(run)
df_allf['fsid'] = sub
df_allf['run'] = runs
df_allf = df_allf.groupby('fsid').mean().reset_index()

In [ ]:
fsid = df_allf.fsid
df_allf = df_allf.iloc[:, 1:]
df_allf['fsid'] = fsid
df_allf.to_csv('../derivatives/childhood/allf.csv', index=False)

In [ ]:
H_child = pd.read_csv('../derivatives/childhood/H_cbpd.csv', header=0)
part_child = pd.read_csv('../derivatives/childhood/n125_long_inc_part_coef_avg_nodewise_avgrunsschaefer400_cbpd.csv')
allf_child = pd.read_csv('../derivatives/childhood/allf.csv')
H_adult = pd.read_csv('../derivatives/adult/H.csv', header=0)
part_adult = pd.read_csv('../derivatives/adult/n125_long_inc_part_coef_avg_nodewise_avgrunsschaefer400.csv')
allf_adult = pd.read_csv('../derivatives/adult/allf.csv')

In [ ]:
H_child = H_child.groupby('H_all401').mean().reset_index()
H_child['record_id'] = [x.replace('sub-', '') for x in H_child['H_all401']]

In [ ]:
normalize = pd.read_csv('../dataset/childhood/dat/normalize.csv')
unnormalize = pd.read_csv('../dataset/childhood/dat/nonormalize.csv')
H_child = pd.merge(H_child, normalize, on='record_id')
H_child = H_child.iloc[:, 1:].sort_values(['record_id']).reset_index(drop=True)

In [10]:
part_child = pd.merge(part_child, normalize, left_on='Var1', right_on='record_id')
part_child = part_child.iloc[:, 1:].sort_values(['record_id']).reset_index(drop=True)
part_adult = part_adult.iloc[:, 1:]

In [11]:
allf_child = allf_child.groupby('fsid').mean().reset_index()
allf_child = pd.merge(allf_child, normalize, left_on='fsid', right_on='record_id')
allf_child = allf_child.iloc[:, 1:].sort_values(['record_id']).reset_index(drop=True)

In [12]:
mean = H_child.iloc[:, :400].mean(axis=1)
H_child['mean'] = mean
part_child = part_child[H_child['mean']<1.6].reset_index(drop=True)
allf_child = allf_child[H_child['mean']<1.6].reset_index(drop=True)
H_child = H_child[H_child['mean']<1.6].reset_index(drop=True)

In [13]:
concat_child = pd.DataFrame([SAaxis['finalrank.wholebrain'].reset_index(drop=True), part_child.iloc[:, :400].mean().values, H_child.iloc[:, :400].mean().values, allf_child.iloc[:, :400].mean().values]).T
concat_child.columns = ['SArank', 'part', 'H', 'allf']
concat_child.to_csv('../derivatives/childhood/original.csv', index=False)

In [14]:
concat_adult = pd.DataFrame([SAaxis['finalrank.wholebrain'].reset_index(drop=True), part_adult.iloc[:, :400].mean().values, H_adult.iloc[:, :400].mean().values, allf_adult.iloc[:, :400].mean().values]).T
concat_adult.columns = ['SArank', 'part', 'H', 'allf']
concat_adult.to_csv('../derivatives/adult/original.csv', index=False)

In [15]:
ages = []
for fsid in H_child.record_id:
    ages.append(unnormalize[unnormalize.record_id==fsid].age_scan.values[0])
H_child['age_scan'] = ages
part_child['age_scan'] = ages

In [16]:
ages = []
for fsid in allf_child.record_id:
    ages.append(unnormalize[unnormalize.record_id==fsid].age_scan.values[0])
allf_child['age_scan'] = ages

In [17]:
H_child.to_csv('../derivatives/childhood/hurst.csv', index=False)
part_child.to_csv('../derivatives/childhood/part.csv', index=False)
allf_child.to_csv('../derivatives/childhood/allf.csv', index=False)

In [18]:
df = pd.read_csv('../dataset/rna/brodmann_schaefer_0.5.csv')
Visual = [x for x in df.columns if 'BA17' in x]
Auditory = [x for x in df.columns if 'BA41' in x]
Sensory =  [x for x in df.columns if 'BA1_3' in x] + [x for x in df.columns if 'BA2' in x]
Motor = [x for x in df.columns if 'BA4' in x]
vPFC = [x for x in df.columns if 'BA44' in x] +  [x for x in df.columns if 'BA45' in x]
dPFC = [x for x in df.columns if 'BA8' in x] + [x for x in df.columns if 'BA9' in x] + [x for x in df.columns if 'BA46' in x]
OFC = [x for x in df.columns if 'BA11' in x] + [x for x in df.columns if 'BA12' in x]+ [x for x in df.columns if 'BA13' in x]  + [x for x in df.columns if 'BA14_' in x]
TEv = [x for x in df.columns if 'BA20' in x]
ACC = [x for x in df.columns if 'BA24' in x] + [x for x in df.columns if 'BA25' in x] + [x for x in df.columns if 'BA32' in x]
STC = [x for x in df.columns if 'BA22' in x]
IPC = [x for x in df.columns if 'BA39' in x] +  [x for x in df.columns if 'BA40' in x]
H_child_brodmann = pd.DataFrame()
H_child_brodmann_mean = pd.DataFrame()
sas = []
for i, x in enumerate([Visual, Auditory, Sensory, Motor, vPFC, dPFC, OFC, TEv, ACC, STC, IPC]):
    included = df.loc[:, x].sum(axis=1)>0
    H_child_brodmann = pd.concat([H_child_brodmann, H_child.iloc[:, df[included==True].index]], axis=1)
    H_child_brodmann_mean = pd.concat([H_child_brodmann_mean, H_child.iloc[:, df[included==True].index].mean(axis=1)], axis=1)
    sas += list(np.repeat(i, len(df[included==True].index)))

In [19]:
shcaferbrod = pd.DataFrame(df.region)
shcaferbrod['group'] = np.zeros(400)
for i, x in enumerate([Visual, Auditory, Sensory, Motor, IPC, STC, vPFC, dPFC, OFC, TEv, ACC]):
    included = df.loc[:, x].sum(axis=1)>0
    shcaferbrod['included'] = included
    shcaferbrod.loc[shcaferbrod.included==True, 'group']=i+1
    print(shcaferbrod[shcaferbrod.included==True].index)
shcaferbrod.iloc[:, :-1].to_csv('../dataset/rna/brodmann_schafer_clustering.csv', index=False)

Int64Index([205, 206], dtype='int64')
Int64Index([44], dtype='int64')
Int64Index([52, 53, 72, 73, 85, 108, 116, 133, 134, 168, 171, 189, 316, 335,
            362],
           dtype='int64')
Int64Index([44, 86, 103, 105, 109, 173, 182, 247, 309, 313], dtype='int64')
Int64Index([86, 172, 173, 187], dtype='int64')
Int64Index([171], dtype='int64')
Int64Index([105, 309], dtype='int64')
Int64Index([349], dtype='int64')
Int64Index([88, 314], dtype='int64')
Int64Index([116], dtype='int64')
Int64Index([108, 316], dtype='int64')


In [20]:
shcaferbrod[shcaferbrod.group>0].shape

(32, 3)

In [21]:
sa = pd.DataFrame(sas)
sa.columns = ['SArank']
sa.to_csv('../derivatives/childhood/brodmann_sa.csv', index=False)

In [22]:
H_child_brodmann['age_scan'] = H_child.age_scan
H_child_brodmann['male'] = H_child.male
H_child_brodmann['t1_rating_avg'] = H_child.t1_rating_avg
H_child_brodmann_mean['age_scan'] = H_child.age_scan
H_child_brodmann_mean['male'] = H_child.male
H_child_brodmann_mean['t1_rating_avg'] = H_child.t1_rating_avg

In [23]:
H_child_brodmann.to_csv('../derivatives/childhood/hurst_brodmann.csv', index=False)
H_child_brodmann_mean.to_csv('../derivatives/childhood/hurst_brodmann_mean.csv', index=False)

In [24]:
part_child[part_child['age_scan']<9].reset_index(drop=True).to_csv('../derivatives/childhood/part_sensitivity.csv', index=False)
H_child[H_child['age_scan']<9].reset_index(drop=True).to_csv('../derivatives/childhood/hurst_sensitivity.csv', index=False)
allf_child[allf_child['age_scan']<9].reset_index(drop=True).to_csv('../derivatives/childhood/allf_sensitivity.csv', index=False)

sa = SAaxis.sort_values('finalrank.wholebrain').index.values
H_sort = pd.DataFrame()
for x in sa:
    H_sort = pd.concat([H_sort, H_child.iloc[:, x]], axis=1)
#H_all = H_sort.to_numpy().flatten()
#H_sort_zscore = pd.DataFrame(stats.zscore(H_all).reshape(H_sort.shape))
H_sort_zscore = stats.zscore(H_sort)
H_sort_zscore.columns = H_sort.columns
H_sort_zscore['fsid'] = H_child['fsid']
H_sort_zscore['age_scan'] = H_child['age_scan']
H_sort_zscore['age_square_scan'] = H_child['age_square_scan']
H_sort_zscore['male'] = H_child['male']
H_sort_zscore['t1_rating_avg'] = H_child['t1_rating_avg']
H_sort_zscore.to_csv('../derivatives/childhood/hurst_sorted.csv', index=False)

a = pd.DataFrame([H_sort_zscore.iloc[:, :133].mean(axis=1), H_sort_zscore.iloc[:,:133:266].mean(axis=1), H_sort_zscore.iloc[:, 266:400].mean(axis=1)]).T
a['age_scan'] =  H_child['age_scan']

In [25]:
sa = SAaxis.sort_values('finalrank.wholebrain').index.values
H_sort = pd.DataFrame()
for x in sa:
    H_sort = pd.concat([H_sort, H_child.iloc[:, x]], axis=1)
H_all = H_sort.to_numpy().flatten()
H_sort_zscore = pd.DataFrame(stats.zscore(H_all).reshape(H_sort.shape))
H_sort_zscore = stats.zscore(H_sort)
H_sort_zscore.columns = H_sort.columns
H_sort_zscore['fsid'] = H_child['fsid']
H_sort_zscore['age_scan'] = H_child['age_scan']
H_sort_zscore['age_square_scan'] = H_child['age_square_scan']
H_sort_zscore['male'] = H_child['male']
H_sort_zscore['t1_rating_avg'] = H_child['t1_rating_avg']
H_sort_zscore.to_csv('../derivatives/childhood/hurst_sorted.csv', index=False)

a = pd.DataFrame([H_sort_zscore.iloc[:, :133].mean(axis=1), H_sort_zscore.iloc[:,:133:266].mean(axis=1), H_sort_zscore.iloc[:, 266:400].mean(axis=1)]).T
a['age_scan'] =  H_child['age_scan']

sa = SAaxis.sort_values('finalrank.wholebrain').index.values
part_sort = pd.DataFrame()
for x in sa:
    part_sort = pd.concat([part_sort, part_child.iloc[:, x]], axis=1)
part_sort = stats.zscore(part_sort)
part_sort['fsid'] = part_child['fsid']
part_sort['age_scan'] = part_child['age_scan']
part_sort['age_square_scan'] = part_child['age_square_scan']
part_sort['male'] = part_child['male']
part_sort['t1_rating_avg'] = part_child['t1_rating_avg']
part_sort.to_csv('../derivatives/childhood/part_sorted.csv', index=False)

In [26]:
sa = SAaxis.sort_values('finalrank.wholebrain').index.values
part_sort = pd.DataFrame()
for x in sa:
    part_sort = pd.concat([part_sort, part_child.iloc[:, x]], axis=1)
H_all = part_sort.to_numpy().flatten()
part_sort_zscore = pd.DataFrame(stats.zscore(H_all).reshape(part_sort.shape))
part_sort_zscore = stats.zscore(part_sort)
part_sort_zscore.columns = part_sort.columns
part_sort_zscore['fsid'] = H_child['fsid']
part_sort_zscore['age_scan'] = H_child['age_scan']
part_sort_zscore['age_square_scan'] = H_child['age_square_scan']
part_sort_zscore['male'] = H_child['male']
part_sort_zscore['t1_rating_avg'] = H_child['t1_rating_avg']
part_sort_zscore.to_csv('../derivatives/childhood/part_sorted.csv', index=False)

In [27]:
sa = SAaxis.sort_values('finalrank.wholebrain').index.values
allf_sort = pd.DataFrame()
for x in sa:
    allf_sort = pd.concat([allf_sort, allf_child.iloc[:, x]], axis=1)
H_all = allf_sort.to_numpy().flatten()
allf_sort_zscore = pd.DataFrame(stats.zscore(H_all).reshape(allf_sort.shape))
allf_sort_zscore = stats.zscore(allf_sort)
allf_sort_zscore.columns = allf_sort.columns
allf_sort_zscore['fsid'] = allf_child['fsid_y']
allf_sort_zscore['age_scan'] = allf_child['age_scan']
allf_sort_zscore['age_square_scan'] = allf_child['age_square_scan']
allf_sort_zscore['male'] = allf_child['male']
allf_sort_zscore['t1_rating_avg'] = allf_child['t1_rating_avg']
allf_sort_zscore.to_csv('../derivatives/childhood/allf_sorted.csv', index=False)

allf_sort = pd.DataFrame()
for x in sa:
    allf_sort = pd.concat([allf_sort, allf_child.iloc[:, x]], axis=1)
#allf_sort = stats.zscore(part_sort)
allf_sort['fsid'] = allf_child['fsid_y']
allf_sort['age_scan'] = allf_child['age_scan']
allf_sort['age_square_scan'] = allf_child['age_square_scan']
allf_sort['male'] = allf_child['male']
allf_sort['t1_rating_avg'] = allf_child['t1_rating_avg']
allf_sort.to_csv('../derivatives/childhood/allf_sorted.csv', index=False)

In [28]:
H_sort_zscore[H_sort_zscore['age_scan']<8].reset_index(drop=True).to_csv('../derivatives/childhood/hurst_sorted_sensitivity.csv')
part_sort_zscore[part_sort_zscore['age_scan']<8].reset_index(drop=True).to_csv('../derivatives/childhood/part_sorted_sensitivity.csv')
allf_sort_zscore[allf_sort_zscore['age_scan']<8].reset_index(drop=True).to_csv('../derivatives/childhood/allf_sorted_sensitivity.csv')

In [30]:
concat_child = pd.DataFrame([part_child.iloc[:, :400].mean(axis=1).values, allf_child.iloc[:, :400].mean(axis=1).values, H_child.iloc[:, :400].mean(axis=1).values, part_child.age_scan.values]).T
concat_child.columns = ['part', 'allf',  'H', 'age']
concat_child.sort_values('age').reset_index(drop=True).to_csv('../derivatives/childhood/indv.csv', index=False)

In [31]:
H_income = H_child.dropna(subset=['income_median']).sort_values('income_median').reset_index(drop=True)
H_income.iloc[:int(H_income.shape[0]/2), :].to_csv('../derivatives/childhood/H_income_low.csv', index=False)
H_income.iloc[int(H_income.shape[0]/2):, :].to_csv('../derivatives/childhood/H_income_high.csv', index=False)
part_income = part_child.dropna(subset=['income_median']).sort_values('income_median').reset_index(drop=True)
part_income.iloc[:int(H_income.shape[0]/2), :].to_csv('../derivatives/childhood/part_income_low.csv', index=False)
part_income.iloc[int(H_income.shape[0]/2):, :].to_csv('../derivatives/childhood/part_income_high.csv', index=False)

In [32]:
H_income_sensitivity = H_income[H_income.age_scan<9]
H_income_sensitivity.iloc[:int(H_income_sensitivity.shape[0]/2), :].to_csv('../derivatives/childhood/H_income_sensitivity_low.csv', index=False)
H_income_sensitivity.iloc[int(H_income_sensitivity.shape[0]/2):, :].to_csv('../derivatives/childhood/H_income_sensitivity_high.csv', index=False)
part_income_sensitivity = part_income[part_income.age_scan<9]
part_income_sensitivity.iloc[:int(H_income_sensitivity.shape[0]/2), :].to_csv('../derivatives/childhood/part_income_sensitivity_low.csv', index=False)
part_income_sensitivity.iloc[int(H_income_sensitivity.shape[0]/2):, :].to_csv('../derivatives/childhood/part_income_sensitivity_high.csv', index=False)

In [33]:
brainmap = pd.DataFrame([np.zeros(400), np.zeros(400), np.zeros(400), np.zeros(400)]).T

In [34]:
brainmap.loc[SAaxis.sort_values('finalrank.wholebrain').index[:133], 0]=1
brainmap.loc[SAaxis.sort_values('finalrank.wholebrain').index[133:266], 1]=1
brainmap.loc[SAaxis.sort_values('finalrank.wholebrain').index[266:], 2]=1
brainmap.loc[SAaxis.sort_values('finalrank.wholebrain').index[266:], 2]=1
brainmap['X3'] = SAaxis['finalrank.wholebrain']

In [35]:
brainmap.to_csv('../derivatives/childhood/brainmap.csv', index=False)